# 02. 构建资源检索模型并预测

In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv, GATConv
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
import networkx as nx
import numpy as np
from sklearn.preprocessing import LabelEncoder
import json
import os
import sys


print(f'torch version: {torch.__version__}')
print(f'device: {torch.device("cuda" if torch.cuda.is_available() else "cpu")}')
print(f'python version: {sys.version}')


torch version: 2.3.1.post300
device: cuda
python version: 3.10.16 (main, Dec 11 2024, 16:24:50) [GCC 11.2.0]


## 1. 构建GNN基础模型

In [50]:
class ResourceGNN(nn.Module):
    def __init__(self, num_node_features, hidden_dim=64):
        super(ResourceGNN, self).__init__()
        self.hidden_dim = hidden_dim
        
        # 节点嵌入层 - 使用nn.Embedding而不是直接处理
        self.node_embedding = nn.Embedding(num_node_features, hidden_dim)
        
        # GNN层 - 修改输入输出维度
        self.conv1 = GATConv(in_channels=hidden_dim, 
                            out_channels=hidden_dim, 
                            heads=4, 
                            concat=True,
                            dropout=0.2)
        
        self.conv2 = GATConv(in_channels=hidden_dim*4,  # 因为第一层有4个头
                            out_channels=hidden_dim,
                            heads=1,
                            concat=False,
                            dropout=0.2)
        
        # 添加批标准化
        self.batch_norm1 = nn.BatchNorm1d(hidden_dim * 4)
        self.batch_norm2 = nn.BatchNorm1d(hidden_dim)

    def forward(self, data):
        # 确保输入是在正确的设备上
        x, edge_index = data.x, data.edge_index
        
        # 节点特征嵌入
        x = self.node_embedding(x)  # [num_nodes, hidden_dim]
        
        # 第一层图注意力
        x = self.conv1(x, edge_index)  # [num_nodes, hidden_dim * 4]
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.2, training=self.training)
        
        # 第二层图注意力
        x = self.conv2(x, edge_index)  # [num_nodes, hidden_dim]
        x = self.batch_norm2(x)
        x = F.relu(x)
        
        return x

# 打印模型结构
print(ResourceGNN(num_node_features=10))


ResourceGNN(
  (node_embedding): Embedding(10, 64)
  (conv1): GATConv(64, 64, heads=4)
  (conv2): GATConv(256, 64, heads=1)
  (batch_norm1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [51]:
# 数据预处理模型

class ResourceDataset(Dataset):
    def __init__(self, graph_data, node_types):
        super(ResourceDataset, self).__init__()
        self.graph_data = graph_data
        self.node_types = node_types
        
        # 编码节点类型
        self.type_encoder = LabelEncoder()
        self.type_encoder.fit(list(node_types))
        
        # 构建训练数据
        self.data_list = self._prepare_data()
        
    def _prepare_data(self):
        # 将NetworkX图转换为PyG数据
        G = nx.Graph(self.graph_data)
        
        # 节点特征
        nodes = list(G.nodes())
        node_features = []
        node_mapping = {node: idx for idx, node in enumerate(nodes)}
        
        for node in nodes:
            node_type = G.nodes[node]['type']
            node_features.append(self.type_encoder.transform([node_type])[0])
            
        # 边特征
        edge_index = []
        for src, dst in G.edges():
            edge_index.append([node_mapping[src], node_mapping[dst]])
            edge_index.append([node_mapping[dst], node_mapping[src]])  # 无向图
            
        # 转换为PyTorch张量
        x = torch.tensor(node_features, dtype=torch.long)
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        
        # 创建PyG数据对象
        data = Data(x=x, 
                   edge_index=edge_index, 
                   num_nodes=len(nodes))
        
        return [data]
    
    def len(self):
        return len(self.data_list)
    
    def get(self, idx):
        return self.data_list[idx]

In [52]:
class ResourceSearchModelGNN:
    def __init__(self, model_path=None):
        # 加载图数据
        self.graph_data = self._load_graph_data()
        
        # 获取节点类型
        self.node_types = set()
        for _, attr in self.graph_data.nodes(data=True):
            self.node_types.add(attr['type'])
        
        # 创建数据集
        self.dataset = ResourceDataset(self.graph_data, self.node_types)
        
        # 创建模型
        self.model = ResourceGNN(
            num_node_features=len(self.node_types),
            num_edge_features=1,
            hidden_dim=64
        )
        
        # 加载预训练模型
        if model_path:
            self.model.load_state_dict(torch.load(model_path))
        
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = self.model.to(self.device)
        
    def _load_graph_data(self):
        """加载图数据"""
        with open('../datasets/results/cross_layer_mapping.json', 'r') as f:
            data = json.load(f)
        # 构建NetworkX图
        G = nx.Graph()
        # ... 构建图的代码 ...
        return G
    
    
    def train(self, epochs=100, batch_size=32):
        """训练模型"""
        self.model.train()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
        loader = DataLoader(self.dataset, batch_size=batch_size, shuffle=True)
        
        for epoch in range(epochs):
            total_loss = 0
            for batch in loader:
                batch = batch.to(self.device)
                optimizer.zero_grad()
                
                # 前向传播
                out = self.model(batch.x, batch.edge_index)
                
                # 计算损失（这里使用示例损失函数）
                loss = self._compute_loss(out, batch)
                
                # 反向传播
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
                
            print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(loader):.4f}')
            
            
    def _compute_loss(self, pred, batch):
            """计算损失函数"""
        # 这里实现具体的损失函数
        # 可以根据任务需求设计不同的损失函数
            pass
    
    def save_model(self, path):
        """保存模型"""
        torch.save(self.model.state_dict(), path)
    
    def search(self, query_node: str, node_type: str):
        """使用训练好的模型进行资源检索"""
        self.model.eval()
        
        # 将查询节点转换为模型输入格式
        node_idx = list(self.graph_data.nodes()).index(query_node)
        data = self.dataset[0].to(self.device)
        
        with torch.no_grad():
            # 获取节点嵌入
            node_embeddings = self.model(data.x, data.edge_index)
            query_embedding = node_embeddings[node_idx]
            
            # 计算与其他节点的相似度
            similarities = F.cosine_similarity(
                query_embedding.unsqueeze(0),
                node_embeddings,
                dim=1
            )
            
            # 获取最相关的节点
            top_k = 10
            top_indices = similarities.topk(top_k).indices.cpu().numpy()
            
            # 构建子图
            related_nodes = [list(self.graph_data.nodes())[i] for i in top_indices]
            subgraph = self.graph_data.subgraph(related_nodes)
            
            # 构建结果字符串
            result_str = self._build_result_string(query_node, related_nodes)
            
            return result_str, subgraph
        
    def _build_result_string(self, query_node, related_nodes):
            """构建结果字符串"""
        # ... 实现结果字符串构建逻辑 ...
            pass
    
    def visualize_subgraph(self, subgraph):
        """可视化子图"""
        # ... 实现子图可视化逻辑 ...
        pass

In [53]:
# 训练代码修改
def train_model(model, dataset, device, epochs=100):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    
    for epoch in range(epochs):
        total_loss = 0
        
        for data in DataLoader(dataset, batch_size=1):  # 由于是单个图，batch_size=1
            data = data.to(device)
            optimizer.zero_grad()
            
            # 前向传播
            out = model(data)
            
            # 这里使用重构损失作为示例
            # 可以根据具体任务修改损失函数
            loss = F.mse_loss(out, torch.zeros_like(out))  # 示例损失
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1:03d}, Loss: {total_loss:.4f}')

## 2. 加载数据进行训练


### 2.1 加载数据

In [63]:
def load_graph_data():
    """加载所有图数据并合并成一个完整的图"""
    base_path = '../datasets/results/'
    
    # 加载三个JSON文件
    with open(os.path.join(base_path, 'tenant_graph_dict.json'), 'r', encoding='utf-8') as f:
        virtual_graph = json.load(f)
    
    with open(os.path.join(base_path, 'ha_graph_dict.json'), 'r', encoding='utf-8') as f:
        physical_graph = json.load(f)
        
    with open(os.path.join(base_path, 'cross_layer_mapping.json'), 'r', encoding='utf-8') as f:
        cross_layer = json.load(f)
    
    # 创建完整的NetworkX图
    G = nx.Graph()
    
    # 添加虚拟资源图的节点和边
    for node in virtual_graph['nodes']:
        # 使用id作为节点标识符
        node_id = node['id']
        # 添加节点及其属性
        G.add_node(node_id, 
                  type=node['type'],
                  node_id=node['node_id'])
    
    # 添加虚拟资源图的边
    for link in virtual_graph['links']:
        G.add_edge(link['source'], 
                  link['target'], 
                  connection_type=link['connection_type'])
    
    # 添加物理资源图的节点和边
    for node in physical_graph['nodes']:
        node_id = node['id']
        if node_id not in G:  # 避免重复添加已存在的节点
            G.add_node(node_id,
                      type=node['type'],
                      node_id=node['node_id'])
            
    for link in physical_graph['links']:
        G.add_edge(link['source'], 
                  link['target'],
                  connection_type=link['connection_type'])
    
    # 添加跨层映射关系
    for mapping in cross_layer['virtual_to_physical']:
        vm = mapping['virtual_layer'].get('virtual_machine')
        host = mapping['physical_layer'].get('host')
        ne = mapping['virtual_layer'].get('network_element')
        ha = mapping['physical_layer'].get('ha')
        tru = mapping['physical_layer'].get('tru')
        
        # 添加虚拟机到主机的映射
        if vm and host:
            if vm not in G:
                G.add_node(vm, type='virtual_machine')
            if host not in G:
                G.add_node(host, type='host')
            G.add_edge(vm, host, connection_type='cross_layer')
        
        # 添加网元到HA的映射
        if ne and ha:
            if ne not in G:
                G.add_node(ne, type='network_element')
            if ha not in G:
                G.add_node(ha, type='ha')
            G.add_edge(ne, ha, connection_type='cross_layer')
            
        # 添加主机到TRU的映射
        if host and tru:
            if tru not in G:
                G.add_node(tru, type='tru')
            G.add_edge(host, tru, connection_type='physical')
    
    # 打印统计信息
    print("\nGraph summary:")
    print(f"Number of nodes: {G.number_of_nodes()}")
    print(f"Number of edges: {G.number_of_edges()}")
    
    # 统计节点类型
    node_types = {}
    for _, attr in G.nodes(data=True):
        node_type = attr.get('type')
        if node_type:
            node_types[node_type] = node_types.get(node_type, 0) + 1
    
    print("\nNode types and counts:")
    for node_type, count in node_types.items():
        print(f"  {node_type}: {count}")
    
    return G



### 2.2 创建PyG数据集

In [55]:
# 创建PyG数据集
class ResourceDataset(Dataset):
    def __init__(self, graph_data):
        super(ResourceDataset, self).__init__()
        self.graph_data = graph_data
        
        # 获取所有节点类型
        self.node_types = set()
        for _, attr in graph_data.nodes(data=True):
            self.node_types.add(attr['type'])
        
        # 编码节点类型
        self.type_encoder = LabelEncoder()
        self.type_encoder.fit(list(self.node_types))
        
        # 构建训练数据
        self.data = self._prepare_data()
        
    def _prepare_data(self):
        # 节点特征
        nodes = list(self.graph_data.nodes())
        node_features = []
        node_mapping = {node: idx for idx, node in enumerate(nodes)}
        
        for node in nodes:
            node_type = self.graph_data.nodes[node]['type']
            type_id = self.type_encoder.transform([node_type])[0]
            node_features.append(type_id)
            
        # 边特征
        edge_index = []
        edge_type = []
        for src, dst, data in self.graph_data.edges(data=True):
            edge_index.append([node_mapping[src], node_mapping[dst]])
            edge_index.append([node_mapping[dst], node_mapping[src]])  # 无向图
            # 边类型特征
            conn_type = data.get('connection_type', 'default')
            edge_type.extend([conn_type, conn_type])  # 双向边
            
        # 转换为PyTorch张量
        x = torch.tensor(node_features, dtype=torch.long)
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        
        # 创建PyG数据对象
        data = Data(
            x=x,
            edge_index=edge_index,
            num_nodes=len(nodes),
            node_mapping=node_mapping,
            reverse_mapping={v: k for k, v in node_mapping.items()}
        )
        
        return data
    
    def len(self):
        return 1  # 只有一个图
    
    def get(self, idx):
        return self.data

### 2.3 构建模型

In [68]:
class ResourceGNN(nn.Module):
    def __init__(self, num_node_features, hidden_dim=64):
        super(ResourceGNN, self).__init__()
        self.hidden_dim = hidden_dim
        
        # 节点嵌入层
        self.node_embedding = nn.Embedding(num_node_features, hidden_dim)
        
        # GNN层
        self.conv1 = GATConv(hidden_dim, hidden_dim, heads=4, dropout=0.2)
        self.conv2 = GATConv(hidden_dim*4, hidden_dim, heads=1, dropout=0.2)
        
        # 批标准化层
        self.batch_norm1 = nn.BatchNorm1d(hidden_dim * 4)
        self.batch_norm2 = nn.BatchNorm1d(hidden_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # 节点特征嵌入
        x = self.node_embedding(x)
        
        # 第一层图注意力
        x = self.conv1(x, edge_index)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.2, training=self.training)
        
        # 第二层图注意力
        x = self.conv2(x, edge_index)
        x = self.batch_norm2(x)
        x = F.relu(x)
        
        return x

In [69]:
def compute_loss(node_embeddings, data):
    """
    计算损失函数 - 这里使用节点分类任务作为示例
    你可以根据具体需求修改这个函数
    """
    # 示例：使用相似节点的embeddings应该相近作为损失
    loss = 0
    for i in range(len(data.edge_index[0])):
        src_idx = data.edge_index[0][i]
        dst_idx = data.edge_index[1][i]
        
        src_emb = node_embeddings[src_idx]
        dst_emb = node_embeddings[dst_idx]
        
        # 计算相连节点的embedding相似度
        similarity = F.cosine_similarity(src_emb.unsqueeze(0), dst_emb.unsqueeze(0))
        loss += 1 - similarity  # 最大化相似度
    
    return loss / len(data.edge_index[0])

In [70]:
def train_model(model, dataset, device, epochs=100):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    
    data = dataset[0].to(device)  # 获取图数据
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        
        # 前向传播
        node_embeddings = model(data)
        
        # 计算损失 - 这里使用节点分类任务作为示例
        # 你可以根据具体需求修改损失函数
        loss = compute_loss(node_embeddings, data)
        
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1:03d}, Loss: {loss.item():.4f}')
    
    return model

### 2.4 开始训练

In [59]:
 # 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [64]:
# 加载图数据
# 使用函数
print("Loading graph data...")
graph_data = load_graph_data()
print(f"Graph loaded: {graph_data.number_of_nodes()} nodes, {graph_data.number_of_edges()} edges")

# 验证图的完整性
print("\nVerifying graph structure:")
print("Sample node attributes:", next(iter(graph_data.nodes(data=True))))
print("Sample edge attributes:", next(iter(graph_data.edges(data=True))))


Loading graph data...

Graph summary:
Number of nodes: 3769
Number of edges: 12939

Node types and counts:
  center: 1
  tenant: 23
  network_element: 86
  virtual_machine: 3034
  ha: 6
  host: 575
  tru: 6
  tor: 32
  eor: 6
Graph loaded: 3769 nodes, 12939 edges

Verifying graph structure:
Sample node attributes: ('xbxa_dc4', {'type': 'center', 'node_id': 0})
Sample edge attributes: ('xbxa_dc4', 'gs_B5G', {'connection_type': 'physical'})


In [65]:
# 创建数据集
print("Creating dataset...")
dataset = ResourceDataset(graph_data)
print(f"Number of node types: {len(dataset.node_types)}")

Creating dataset...
Number of node types: 9


In [67]:
# 创建模型
print("Creating model...")
model = ResourceGNN(
    num_node_features=len(dataset.node_types),
    hidden_dim=64
).to(device)

# 训练模型
print("Starting training...")
model = train_model(model, dataset, device, epochs=100)

Creating model...
Starting training...
Epoch 010, Loss: 0.3208
Epoch 020, Loss: 0.2843
Epoch 030, Loss: 0.2514
Epoch 040, Loss: 0.2323
Epoch 050, Loss: 0.2045
Epoch 060, Loss: 0.1882
Epoch 070, Loss: 0.1765
Epoch 080, Loss: 0.1465
Epoch 090, Loss: 0.1315
Epoch 100, Loss: 0.1273
